In [0]:
dbutils.fs.ls(
   "/Volumes/workspace/default/spotify"
)

In [0]:
caminho_data = "/Volumes/workspace/default/spotify/data.csv"

In [0]:
df_data = spark.read.csv(caminho_data, header=True, inferSchema=True)

In [0]:
df_data.display()

In [0]:
df_data.dtypes

In [0]:
colunas_float = [
    "acousticness",
    "danceability",
    "energy",
    "instrumentalness",
    "liveness",
    "loudness",
    "speechiness",
    "tempo",
    "valence"
]

colunas_int = [
    "year",
    "duration_ms",
    "mode",
    "key",
    "explicit",
    "popularity"
]

In [0]:
from pyspark.sql.functions import col

# listas que você já definiu antes
# colunas_float = ['loudness', 'speechiness', 'tempo', 'valence']
# colunas_int   = ['duration_ms', 'mode', 'key', 'explicit', 'popularity']

# converte colunas para double (float)
for c in colunas_float:
    df_data = df_data.withColumn(c, col(c).cast("double"))

# converte colunas para int
for c in colunas_int:
    df_data = df_data.withColumn(c, col(c).cast("int"))

In [0]:
df_data.dtypes

In [0]:
type(df_data)

In [0]:
from pyspark.sql.functions import expr

# FLOAT/DOUBLE
for c in colunas_float:
    df_data = df_data.withColumn(c, expr(f"try_cast({c} as double)"))

# INT
for c in colunas_int:
    df_data = df_data.withColumn(c, expr(f"try_cast({c} as int)"))

In [0]:
from pyspark.sql.functions import col

for c, t in df_data.dtypes:
    if t in ("double", "int"):  # apenas colunas numéricas
        print("🔎 Checando:", c)
        df_data.filter(col(c).cast("string").rlike("[A-Za-z]")).select(c, "artists", "name").show(5, False)

In [0]:
from pyspark.sql.functions import when

df_data = df_data.withColumn(
    "danceability",
    when(col("danceability").cast("string").contains(texto), None)
    .otherwise(col("danceability"))
)

In [0]:
df_data = df_data.pandas_api()

In [0]:
type(df_data)
df_data.head(5)

In [0]:
df_data.info()

In [0]:
colunas_float = [ 'acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'valence']

colunas_int = ['duration_ms', 'mode', 'key', 'explicit', 'popularity']

In [0]:
df_data.dtypes

In [0]:
df_data[colunas_float] = df_data[colunas_float].astype(float)
df_data[colunas_int] = df_data[colunas_int].astype(int)

In [0]:
df_data.head()

In [0]:
type(df_data.artists.iloc[0])

In [0]:
x = df_data.artists.iloc[0:9]
x

In [0]:
df_data['artists'] = df_data.artists.str.replace("\[|\]|\'","")
df_data['artists'] = df_data.artists.str.replace(",",";")

# Remove colchetes e aspas
#df_data['artists'] = df_data['artists'].str.replace(r"[\[\]']", '', regex=True)

# (Opcional) substitui vírgulas por ponto-e-vírgula se quiser deixar mais legível
#df_data['artists'] = df_data['artists'].str.replace(',', ';', regex=True)

In [0]:
df_data.dtypes

In [0]:
len(df_data)

In [0]:
df_data.head(5)

In [0]:
dbutils.fs.ls('/Volumes/workspace/default/spotify/')

In [0]:
dbutils.fs.mkdirs('/Volumes/workspace/default/spotify/dados_tratados')

In [0]:
dbutils.fs.ls('/Volumes/workspace/default/spotify/')

In [0]:
df_data.dtypes

In [0]:
df_data['artists'] = df_data['artists'].astype(str)
df_data['artists'] = df_data['artists'].str.replace(r"[\[\]']", '', regex=True)
df_data['artists'] = df_data['artists'].str.replace(',', ';', regex=False)

In [0]:
df_com_conteudo = df_data[df_data["artists"].str.strip() != ""]
display(df_com_conteudo.head(10))


In [0]:
df_data.dtypes

In [0]:
import re
import pandas as pd

# Cria coluna com caracteres especiais
df_data["bad_chars"] = df_data["artists"].apply(lambda x: re.sub(r"[a-zA-Z0-9áéíóúÁÉÍÓÚãõÃÕçÇ&;\s]", "", str(x)))

# Filtra apenas linhas que contêm caracteres especiais
df_suja = df_data[df_data["bad_chars"].str.len() > 0]

# Exibe o resultado no Databricks
display(df_suja[["artists", "bad_chars"]])


In [0]:
df_data.dtypes

In [0]:
df_data.to_parquet('/Volumes/workspace/default/spotify/dados_tratados/data.parquet')

In [0]:
## ANALISE DOS DADOS
import pyspark.pandas as ps

In [0]:
path = '/Volumes/workspace/default/spotify/dados_tratados/data.parquet'
df_data = ps.read_parquet(path)
df_data.head()

In [0]:
df_data.describe()

In [0]:
len(df_data.year.unique())

In [0]:
df_data.year.value_counts()

In [0]:
df_data.year.value_counts().sort_index()

In [0]:
df_data.year.value_counts().sort_index().plot.bar

In [0]:
df_data['decade'] = df_data.year.apply(lambda year: f'{(year//10)*10}s')

In [0]:
df_data.head()

In [0]:
df_data_2 = df_data[['decade']]
df_data_2['qtd'] = 1

In [0]:
df_data_2 = df_data_2.groupby('decade').sum()
df_data_2

In [0]:
df_data_2.sort_index().plot.bar(y='qtd')

In [0]:
dbutils.fs.ls(
   "/Volumes/workspace/default/spotify/dados_tratados"
)


In [0]:
## Trabalhando com outros arquivos
df_year = spark.read.csv(
    "/Volumes/workspace/default/spotify/data_by_year.csv",
    header=True,        # se o CSV tem cabeçalho
    inferSchema=True,   # Spark tenta adivinhar os tipos
    sep=","             # separador padrão (pode trocar)
)

In [0]:
df_year.printSchema()
df_year.head()

In [0]:
df_pd = df_year.select("year", "duration_ms").toPandas()

df_pd.plot.line(x="year", y="duration_ms")

In [0]:
df_pd = df_year.select(
    "year",
    "acousticness",
    "danceability",
    "energy",
    "instrumentalness",
    "liveness",
    "speechiness",
    "valence"
).toPandas()

df_pd.plot.line(
    x="year",
    y=["acousticness", "danceability", "energy",
       "instrumentalness", "liveness", "speechiness", "valence"]
)

In [0]:
dbutils.fs.ls(
   "/Volumes/workspace/default/spotify"
)

In [0]:
## Trabalhando com outros arquivos
df_genres = spark.read.csv(
    "/Volumes/workspace/default/spotify/data_by_genres.csv",
    header=True,        # se o CSV tem cabeçalho
    inferSchema=True,   # Spark tenta adivinhar os tipos
    sep=","             # separador padrão (pode trocar)
)
df_artists = spark.read.csv(
    "/Volumes/workspace/default/spotify/data_by_artist.csv",
    header=True,        # se o CSV tem cabeçalho
    inferSchema=True,   # Spark tenta adivinhar os tipos
    sep=","             # separador padrão (pode trocar)
)
df_Wgenres = spark.read.csv(
    "/Volumes/workspace/default/spotify/data_w_genres.csv",
    header=True,        # se o CSV tem cabeçalho
    inferSchema=True    # Spark tenta adivinhar os tipos
)

In [0]:
df_artists.head()

In [0]:
df_artists = df_artists.toPandas()

In [0]:
artista_ordenado = df_artists.sort_values(by="count", ascending = False)
artista_ordenado.head()

In [0]:
top_artista = artista_ordenado.iloc[0:10]
top_artista

In [0]:
plot_title = 'Top 10 Artistas'
top_artista.plot.barh(y='count', x='artists', title=plot_title)

In [0]:
lista_artistas = top_artista.artists.unique().tolist()
lista_artistas

In [0]:
df_Wgenres = df_Wgenres.toPandas()
df_Wgenres.head()

In [0]:
artista_genero = df_Wgenres.loc[df_Wgenres['artists'].isin(lista_artistas)]
artista_genero = artista_genero[['genres','artists']]
display(artista_genero)

In [0]:
#Salvar todos os arquivos para a pasta de transformados em parquet
